<a href="https://colab.research.google.com/github/ameya1252/5173_HW3/blob/main/hw3_3layers_wonorm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
 from google.colab import files

# Upload a zip file containing data folders
uploaded = files.upload()


Saving Data.zip to Data.zip


In [ ]:
import tensorflow as tf
from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, UpSampling2D, concatenate, BatchNormalization, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import numpy as np
import matplotlib.pyplot as plt
import os
import zipfile
from tensorflow.keras import backend as K

In [ ]:

# Specify the name of the uploaded zip file
zip_filename = "Data.zip"

# Extract the contents to a specific directory (e.g., /content/data)
extract_path = "/content/data"

# Unzip the file
with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)


In [ ]:
BASE_PATH = "/content/data/Data"

def load_data(data_type="train"):
    images = []
    masks = []

    # Paths
    image_path = os.path.join(BASE_PATH, data_type, "image")
    mask_path = os.path.join(BASE_PATH, data_type, "mask")

    for filename in sorted(os.listdir(image_path)):
        if not filename.endswith('.png'):  # Adjust if your images have a different extension
            continue

        # Read Image
        img = tf.io.read_file(os.path.join(image_path, filename))
        img = tf.image.decode_png(img, channels=1)
        images.append(img)

        # Read corresponding mask
        mask = tf.io.read_file(os.path.join(mask_path, filename))
        mask = tf.image.decode_png(mask, channels=1)
        masks.append(mask)

    return tf.stack(images), tf.stack(masks)

# Load Training and Test Data
train_images, train_masks = load_data(data_type="train")
test_images, test_masks = load_data(data_type="test")


In [ ]:
# U-Net architecture with improvements
def build_unet(input_shape, num_layers=2):
    inputs = Input(input_shape)
    x = inputs
    down_layers = []
    for i in range(num_layers):
        x = Conv2D(64 * (2**i), (3, 3), activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = Conv2D(64 * (2**i), (3, 3), activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        if i < num_layers - 1:
            x = Dropout(0.5)(x)
        down_layers.append(x)
        x = MaxPooling2D((2, 2), padding='same')(x)

    x = Dropout(0.5)(x)

    for i in range(num_layers - 1, -1, -1):
        x = UpSampling2D((2, 2))(x)
        x = concatenate([x, down_layers[i]])
        x = Conv2D(64 * (2**i), (3, 3), activation='relu', padding='same')(x)
        x = BatchNormalization()(x)
        x = Conv2D(64 * (2**i), (3, 3), activation='relu', padding='same')(x)
        x = BatchNormalization()(x)

    outputs = Conv2D(1, (1, 1), activation='sigmoid')(x)
    return Model(inputs=inputs, outputs=outputs)

In [ ]:
def dice_coef(y_true, y_pred):
    smooth = 1e-5
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) + smooth)

def iou_coef(y_true, y_pred):
    smooth = 1e-5
    y_true_f = tf.keras.backend.flatten(y_true)
    y_pred_f = tf.keras.backend.flatten(y_pred)
    intersection = tf.keras.backend.sum(y_true_f * y_pred_f)
    union = tf.keras.backend.sum(y_true_f) + tf.keras.backend.sum(y_pred_f) - intersection
    return (intersection + smooth) / (union + smooth)


In [ ]:
def tversky_loss(y_true, y_pred, alpha=0.5, beta=0.5):
    smooth = 1e-5
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    true_positives = K.sum(y_true_f * y_pred_f)
    false_positives = K.sum((1 - y_true_f) * y_pred_f)
    false_negatives = K.sum(y_true_f * (1 - y_pred_f))
    tversky = (true_positives + smooth) / (true_positives + alpha * false_positives + beta * false_negatives + smooth)
    return 1 - K.mean(tversky)

In [ ]:
def resize_images(images, target_height, target_width):
    resized_images = []
    for img in images:
        img = tf.image.resize(img, (target_height, target_width))
        resized_images.append(img)
    return tf.stack(resized_images)

# Resize Training and Test Images
target_height = 256  # Update according to your desired image height
target_width = 256   # Update according to your desired image width

resized_train_images = resize_images(train_images, target_height, target_width)
resized_test_images = resize_images(test_images, target_height, target_width)

# Resize Training and Test Masks
resized_train_masks = resize_images(train_masks, target_height, target_width)
resized_test_masks = resize_images(test_masks, target_height, target_width)



# Update Input Shape
input_shape = (target_height, target_width, 1)

# Build and Compile Model
model = build_unet(input_shape, num_layers=3)
model.compile(optimizer=Adam(learning_rate=0.000000001), loss=tversky_loss, metrics=[dice_coef, iou_coef])

# Train Model
model.fit(resized_train_images, resized_train_masks, batch_size=8, epochs=10, validation_split=0.1)


Epoch 1/10
9/9 [==============================] - 12s 560ms/step - loss: 0.2114 - dice_coef: 0.7886 - iou_coef: 0.6511 - val_loss: 0.5215 - val_dice_coef: 0.4785 - val_iou_coef: 0.3145
Epoch 2/10
9/9 [==============================] - 5s 514ms/step - loss: 0.2095 - dice_coef: 0.7905 - iou_coef: 0.6536 - val_loss: 0.4396 - val_dice_coef: 0.5604 - val_iou_coef: 0.3893
Epoch 3/10
9/9 [==============================] - 5s 523ms/step - loss: 0.2091 - dice_coef: 0.7909 - iou_coef: 0.6541 - val_loss: 0.4010 - val_dice_coef: 0.5990 - val_iou_coef: 0.4275
Epoch 4/10
9/9 [==============================] - 5s 523ms/step - loss: 0.2101 - dice_coef: 0.7899 - iou_coef: 0.6528 - val_loss: 0.3711 - val_dice_coef: 0.6289 - val_iou_coef: 0.4586
Epoch 5/10
9/9 [==============================] - 5s 531ms/step - loss: 0.2108 - dice_coef: 0.7892 - iou_coef: 0.6518 - val_loss: 0.3465 - val_dice_coef: 0.6535 - val_iou_coef: 0.4853
Epoch 6/10
9/9 [==============================] - 5s 538ms/step - loss: 0.2095 

In [ ]:
# Resize Test Images and Masks
resized_test_images = resize_images(test_images, target_height, target_width)
resized_test_masks = resize_images(test_masks, target_height, target_width)



# Predict on Resized Test Images
test_preds = model.predict(resized_test_images)

# Evaluate on Resized Test Masks
dice_score = dice_coef(resized_test_masks, test_preds)
iou_score = iou_coef(resized_test_masks, test_preds)

print(f"2 Layers Model - Dice Score: {dice_score}, IoU Score: {iou_score}")

1/1 [==============================] - 0s 242ms/step
2 Layers Model - Dice Score: 0.6749468445777893, IoU Score: 0.5093734264373779
